In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Cargamos las librerías necesarias
import pandas as pd
import numpy as np

In [3]:
# Cargamos datos con SQR
perfiles_sqr = pd.read_parquet('../data/processed/perfiles_sqr_filtrado_not_filtered.parquet')
# perfiles_sqr = pd.read_parquet('../data/raw/flex_sesiones.parquet')

In [4]:
from io import StringIO

 

data = StringIO("""
1610,2448,,,,,,,,
2445,2988,,,,,,,,
1497,1751,1752,1753,,,,,,
1522,1523,1799,,,,,,,
1798,2397,2992,2996,3905,,,,,
1595,1596,1626,1813,1814,,,,,
8026,8036,8226,8282,,,,,,
7991,7992,8049,,,,,,,
8031,8040,8052,8191,8328,,,,,
8032,8041,8053,8054,8329,,,,,
5519,5547,5554,5638,6133,8024,8027,8037,,
5525,5546,5548,5550,5639,6135,7845,8039,,
5710,5814,6964,,,,,,,
5707,5715,5747,6963,,,,,,
5746,5897,6055,6269,,,,,,
5745,5896,6021,6022,6054,,,,,
5844,5869,5878,5998,6060,6102,6611,6844,6845,7592
7593,6061,6843,6610,5843,6612,5845,5868,,
5888,5996,,,,,,,,
5956,6005,6006,,,,,,,
5964,6275,,,,,,,,
5966,6274,,,,,,,,
6014,6024,6078,,,,,,,
6039,6314,,,,,,,,
6115,6119,6335,,,,,,,
6228,6378,6606,6905,,,,,,
6629,6848,,,,,,,,
6249,6261,7854,,,,,,,
6497,6502,8200,,,,,,,
6569,6570,6756,,,,,,,
6614,6868,7719,8225,,,,,,
6613,6869,7718,,,,,,,
6628,6692,6753,6755,,,,,,
""")
profiles = pd.read_csv(data, header=None)
perfiles_sqr['alejandro_profile'] = None
def f(r):
    perfiles_sqr.loc[perfiles_sqr['idPerfiles'].isin(r.values[~np.isnan(r.values)]), 'alejandro_profile'] = r.name
profiles.apply(f, axis=1)
aux = perfiles_sqr[~perfiles_sqr['alejandro_profile'].isna()]
aux['temporal_profile'] = aux.sort_values('fechaInicio').groupby('alejandro_profile')['presiones'].apply(lambda x: (x.shift() != x).rolling(len(x), min_periods=1).sum())


In [5]:
tabla1 = aux.groupby(['alejandro_profile', 'temporal_profile']).agg({'presiones': np.max, 'sqr': np.mean})

In [6]:
aux[['idPerfiles', 'alejandro_profile']]

,idPerfiles,alejandro_profile
33571,6378,25
33679,6378,25
33751,6335,24
33808,6335,24
33839,6135,11
...,...,...
84790,8052,8
84795,7854,27
84796,7854,27
84797,7854,27


In [7]:
tabla1.reset_index()['alejandro_profile'].value_counts()[tabla1.reset_index()['alejandro_profile'].value_counts() > 1].index

Int64Index([25, 31, 16, 27, 10, 11, 17], dtype='int64')

In [8]:
tabla2 = tabla1.reset_index()[tabla1.reset_index()['alejandro_profile'].isin([25, 31, 16, 27, 10, 11, 17])]

In [9]:
tabla2['configuracion2'] = tabla2['presiones']

In [10]:
tabla2['magnitud'] = tabla2.groupby('alejandro_profile')['sqr'].apply(lambda x: (x - x.shift()))
tabla2['configuracion1'] = tabla2.groupby('alejandro_profile')['presiones'].apply(lambda x: (x.shift()))

In [11]:
tabla2

,alejandro_profile,temporal_profile,presiones,sqr,configuracion2,magnitud,configuracion1
3,10,1.0,122332,63.664021,122332,NaN,NaN
4,10,2.0,012232,39.188148,012232,-24.475873,122332
5,11,1.0,123222,47.930909,123222,NaN,NaN
6,11,2.0,122222,49.554749,122222,1.623840,123222
9,16,1.0,012211,46.807144,012211,NaN,NaN
10,16,2.0,122221,53.876857,122221,7.069713,012211
11,17,1.0,022332,58.332234,022332,NaN,NaN
12,17,2.0,122232,50.334789,122232,-7.997445,022332
18,25,1.0,122332,53.684476,122332,NaN,NaN
19,25,2.0,444444,98.912500,444444,45.228024,122332


In [12]:
tabla3 = tabla2.dropna()[['alejandro_profile', 'configuracion1', 'configuracion2', 'magnitud']]
tabla3['mejora'] = tabla3['magnitud'] > 0

In [13]:
tabla3

,alejandro_profile,configuracion1,configuracion2,magnitud,mejora
4,10,122332,012232,-24.475873,False
6,11,123222,122222,1.623840,True
10,16,012211,122221,7.069713,True
12,17,022332,122232,-7.997445,False
19,25,122332,444444,45.228024,True
20,25,444444,122332,-41.349455,False
22,27,012232,123333,-4.263457,False
27,31,122322,122222,-3.103121,False
